In [643]:
from ortools.sat.python import cp_model

In [644]:
microcycles = 2
days = 10 # 10 days per microcyle
workouts = ['Easy', 'Threshold', 'Rest', 'Long Run']
all_days = range(days)
all_workouts = range(len(workouts)) # get indices from workouts list
all_microcycles = range(microcycles)
# [print(a) for a in all_workouts]

In [645]:
model = cp_model.CpModel()

In [646]:
# create an array of variables
run_schedule = {}
for w in all_workouts:
    for m in all_microcycles:
        for d in all_days:
            run_schedule[(w, m, d)] = model.NewBoolVar(f'w{w}m{m}d{d}')

In [647]:
# Assign exactly one workout to each day
for m in all_microcycles:
    for d in all_days:
        model.AddExactlyOne(run_schedule[(w, m, d)] for w in all_workouts)

In [648]:
# Require 2-3 threshold sessions per microcycle
max_threshold_per_micro = 3
min_threshold_per_micro = 2
for m in all_microcycles:
    num_per_cycle = []
    num_rest_per_cycle = []
    num_long_run_per_cycle = []
    for w in all_workouts:  
        if workouts[w] == 'Threshold':
            for d in all_days:
                if d + 1 < days and d - 1 >= 0:
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m, d + 1)].Not())
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m, d - 1)].Not())
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m, d + 1)])
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m, d - 1)])
                elif d - 1 < 0 and m - 1 >= 0:
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m, d + 1)].Not())
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m - 1, days - 1)].Not())
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m, d + 1)])
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m - 1, days - 1)])
                elif d + 1 >= days and m + 1 < microcycles: 
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m + 1, 0)].Not())
                    model.AddImplication(run_schedule[(w, m, d)], run_schedule[(w, m, d - 1)].Not())
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m + 1, 0)])
                    # model.Add(run_schedule[(w, m, d)] != run_schedule[(w, m, d - 1)])
                num_per_cycle.append(run_schedule[(w, m, d)]) #appends boolean 1's and 0's
            model.Add(sum(num_per_cycle) <= max_threshold_per_micro)
            model.Add(sum(num_per_cycle) >= min_threshold_per_micro)
        elif workouts[w] == 'Rest':
            for d in all_days:
                num_rest_per_cycle.append(run_schedule[(w, m, d)])
            model.Add(sum(num_rest_per_cycle) == 1)
            if d + 1 < days:
                model.Add(run_schedule[(0, m, d + 1)] == True).OnlyEnforceIf(run_schedule[(w, m, d)])
                # model.Add(run_schedule[(p, m, d)] == False)
        elif workouts[w] == 'Long Run':
            for d in all_days:
                num_long_run_per_cycle.append(run_schedule[(w, m, d)])
            model.Add(sum(num_long_run_per_cycle) == 1)


In [649]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

In [650]:
# Register solutions callback
class RunPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, run_schedule, days, microcycles, workouts, solution_limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._run_schedule = run_schedule
        self._days = days
        self._microcycles = microcycles
        self._workouts = workouts
        self._solution_count = 0
        self._solution_limit = solution_limit
    
    def on_solution_callback(self):
        self._solution_count += 1
        for m in range(self._microcycles):
            print(f'm:{m+1}')
            for d in range(self._days): 
                for w in range(len(self._workouts)):
                    if self.Value(self._run_schedule[(w, m, d)]):
                        print(f'day:{d+1} Workout: {self._workouts[w]}')
        if self._solution_count >= self._solution_limit:
            print(f'solution limit reached {self._solution_count}')
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

In [651]:
solution_limit = 1
callback = RunPartialSolutionPrinter(run_schedule, days, microcycles, workouts, solution_limit)

In [652]:
status = solver.Solve(model, callback)
# next constraint want easy day after rest day

m:1
day:1 Workout: Long Run
day:2 Workout: Rest
day:3 Workout: Threshold
day:4 Workout: Easy
day:5 Workout: Threshold
day:6 Workout: Easy
day:7 Workout: Easy
day:8 Workout: Easy
day:9 Workout: Easy
day:10 Workout: Easy
m:2
day:1 Workout: Long Run
day:2 Workout: Rest
day:3 Workout: Threshold
day:4 Workout: Easy
day:5 Workout: Threshold
day:6 Workout: Easy
day:7 Workout: Easy
day:8 Workout: Easy
day:9 Workout: Easy
day:10 Workout: Easy
solution limit reached 1


In [653]:
# solver.StatusName()
print(solver.ResponseStats())

CpSolverResponse summary:
status: FEASIBLE
objective: 0
best_bound: 0
booleans: 80
conflicts: 2
branches: 207
propagations: 681
integer_propagations: 158
restarts: 161
lp_iterations: 0
walltime: 0.0117979
usertime: 0.0117984
deterministic_time: 0.00023218
gap_integral: 0

